In [7]:
import os
from serpapi import SerpApiClient
from dotenv import load_dotenv

load_dotenv()

# --- THIẾT LẬP ---
# Tốt nhất là lưu API key trong biến môi trường để bảo mật
# Hoặc bạn có thể tạm thời gán trực tiếp vào đây để thử nghiệm:
# api_key = "API_KEY_CỦA_BẠN_DÁN_VÀO_ĐÂY" 
api_key = os.getenv("SERPAPI_KEY") # Cách làm an toàn hơn

if not api_key:
    raise ValueError("Vui lòng đặt SERPAPI_KEY trong biến môi trường của bạn.")
if not api_key:
    raise ValueError("Vui lòng đặt SERPAPI_KEY trong biến môi trường của bạn.")

# --- CẤU HÌNH TRUY VẤN ---
params = {
    "api_key": api_key,
    "engine": "google",
    "q": "các trường của đại học bách khoa hà nội", # Câu truy vấn của bạn
    "gl": "vn",
    "hl": "vi",
    "num": "5" # Lấy 10 kết quả đầu tiên
}

# --- GỬI YÊU CẦU VÀ XỬ LÝ KẾT QUẢ ---
try:
    print(f"Đang gửi truy vấn: '{params['q']}'...")
    client = SerpApiClient(params)
    results = client.get_dict()

    # --- TRÍCH XUẤT DANH SÁCH KẾT QUẢ TÌM KIẾM THÔNG THƯỜNG ---
    
    # Kiểm tra xem key 'organic_results' có tồn tại trong kết quả không
    if "organic_results" in results:
        print("\n--- CÁC KẾT QUẢ TÌM KIẾM TỰ NHIÊN ---")
        
        # Lặp qua từng kết quả trong danh sách 'organic_results'
        for result in results["organic_results"]:
            position = result.get("position", "N/A")
            title = result.get("title", "N/A")
            link = result.get("link", "N/A")
            snippet = result.get("snippet", "N/A").replace("\n", " ") # Xóa ký tự xuống dòng trong snippet

            print(f"\n[{position}] {title}")
            print(f"   Link: {link}")
            print(f"   Mô tả: {snippet}")
            print("-" * 20)
            
    else:
        print("Không tìm thấy kết quả tìm kiếm tự nhiên (organic results).")
        # In ra toàn bộ kết quả để debug nếu cần
        # print(json.dumps(results, indent=2, ensure_ascii=False))

except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")


Đang gửi truy vấn: 'các trường của đại học bách khoa hà nội'...

--- CÁC KẾT QUẢ TÌM KIẾM TỰ NHIÊN ---

[1] Đại học Bách khoa Hà Nội chính thức có 5 trường trực thuộc
   Link: https://tuoitre.vn/dai-hoc-bach-khoa-ha-noi-chinh-thuc-co-5-truong-truc-thuoc-2023042207374893.htm
   Mô tả: Với việc thành lập hai trường trực thuộc mới, Đại học Bách khoa Hà Nội sẽ có 5 trường trực thuộc gồm Trường Cơ khí, Trường Điện - điện tử và ...
--------------------

[2] Trường - Khoa - Viện đào tạo
   Link: https://hust.edu.vn/vi/co-cau-to-chuc-bai-viet/truong-khoa-vien-dao-tao.html
   Mô tả: Khoa Giáo dục Quốc phòng & An ninh · Khoa Giáo dục Thể chất · Khoa Lý luận Chính trị · Trường Cơ khí · Trường Công nghệ Thông tin và Truyền thông.
--------------------

[3] Đại học Bách khoa Hà Nội thành lập thêm trường thứ 6
   Link: https://giaoduc.net.vn/dai-hoc-bach-khoa-ha-noi-thanh-lap-them-truong-thu-6-post244551.gd
   Mô tả: 6 trường thuộc Đại học Bách khoa Hà Nội bao gồm: Trường Cơ khí; Trường Điện - Điện t

In [14]:
import os
from serpapi import SerpApiClient
from langchain_community.document_loaders import WebBaseLoader

# --- HÀM SCRAPE NỘI DUNG TỪ MỘT URL (Không thay đổi) ---
def scrape_content_from_url(url: str) -> str:
    """
    Sử dụng WebBaseLoader của LangChain để lấy nội dung chính của một trang web.
    Trả về nội dung dưới dạng văn bản thuần túy.
    """
    try:
        print(f"   -> Đang scrape url: {url}")
        loader = WebBaseLoader(url)
        docs = loader.load()
        content = " ".join([doc.page_content for doc in docs])
        cleaned_content = ' '.join(content.split())
        return cleaned_content
    except Exception as e:
        print(f"   -> Lỗi khi scrape url {url}: {e}")
        return ""

# --- HÀM CHÍNH (Đã cập nhật logic lọc) ---
def get_full_context_for_query(query: str, num_results: int = 3): # Tăng num_results mặc định lên 3
    """
    Thực hiện tìm kiếm Google, lọc bỏ domain không mong muốn, 
    sau đó scrape nội dung từ các kết quả hợp lệ.
    """
    api_key = os.getenv("SERPAPI_KEY")
    if not api_key:
        raise ValueError("Vui lòng đặt SERPAPI_KEY trong biến môi trường của bạn.")

    # === BƯỚC 1: TÌM URL BẰNG SERPAPI ===
    print(f"Đang tìm kiếm các trang web liên quan cho: '{query}'...")
    params = {
        "api_key": api_key,
        "engine": "google",
        "q": query,
        "gl": "vn",
        "hl": "vi",
    }
    try:
        client = SerpApiClient(params)
        results = client.get_dict()
        
        if "organic_results" not in results:
            print("Không tìm thấy kết quả tự nhiên.")
            return

        # === LOGIC LỌC ĐÃ ĐƯỢC THÊM VÀO ĐÂY ===
        print("Đang lọc kết quả, loại bỏ các trang từ 'hust.edu.vn'...")
        
        # 1. Lấy toàn bộ danh sách kết quả
        all_organic_results = results["organic_results"]
        
        # 2. Tạo một danh sách link mới, chỉ chứa các link KHÔNG thuộc hust.edu.vn
        filtered_links = [
            result["link"] 
            for result in all_organic_results 
            if "hust.edu.vn" not in result["link"]
        ]
        
        # 3. Lấy số lượng link mong muốn từ danh sách đã lọc
        urls_to_scrape = filtered_links[:num_results]
        
        if not urls_to_scrape:
            print("Không tìm thấy link nào phù hợp sau khi đã lọc.")
            return
            
        print(f"Đã tìm thấy {len(urls_to_scrape)} link phù hợp để lấy dữ liệu.")
        
    except Exception as e:
        print(f"Lỗi khi gọi SerpApi: {e}")
        return

    # === BƯỚC 2: SCRAPE NỘI DUNG TỪ CÁC URL ĐÃ LỌC ===
    print("\nBắt đầu quá trình lấy nội dung chi tiết từ các trang web...")
    full_context = ""
    for i, url in enumerate(urls_to_scrape):
        print(f"Đang xử lý link {i+1}/{len(urls_to_scrape)}")
        content = scrape_content_from_url(url)
        if content:
            full_context += f"--- Nguồn {i+1} (từ {url}) ---\n"
            full_context += content + "\n\n"
    
    print("--- Hoàn thành việc lấy dữ liệu thô! ---")
    return full_context


In [15]:
search_query = "các trường của đại học bách khoa hà nội"
combined_text = get_full_context_for_query(search_query, num_results=1)
print(combined_text)

Đang tìm kiếm các trang web liên quan cho: 'các trường của đại học bách khoa hà nội'...
Đang lọc kết quả, loại bỏ các trang từ 'hust.edu.vn'...
Đã tìm thấy 1 link phù hợp để lấy dữ liệu.

Bắt đầu quá trình lấy nội dung chi tiết từ các trang web...
Đang xử lý link 1/1
   -> Đang scrape url: https://tuoitre.vn/dai-hoc-bach-khoa-ha-noi-chinh-thuc-co-5-truong-truc-thuoc-2023042207374893.htm
--- Hoàn thành việc lấy dữ liệu thô! ---
--- Nguồn 1 (từ https://tuoitre.vn/dai-hoc-bach-khoa-ha-noi-chinh-thuc-co-5-truong-truc-thuoc-2023042207374893.htm) ---
Đại học Bách khoa Hà Nội chính thức có 5 trường trực thuộc - Tuổi Trẻ Online Podcast YouTube Cần biết Rao vặt Xin chào, Cài đặt tài khoản Tin đã lưu Bình luận của bạn Lịch sử giao dịch Dành cho bạn Vào Tuổi Trẻ Sao Thoát Tuổi Trẻ Sao Đăng xuất Đặt báo Đăng ký Tuổi Trẻ Sao Vào trang Tuổi Trẻ Sao Video Thời sự Thế giới Pháp luật Kinh doanh Công nghệ Xe Du lịch Nhịp sống trẻ Văn hóa Giải trí Thể thao Giáo dục Nhà đất Sức khỏe Giả thật Bạn đọc THỜ